In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
diamonds = sns.load_dataset('diamonds')

In [ ]:
diamonds.describe()

In [ ]:
diamonds['cut'].mean

In [ ]:
diamonds['cut'].value_counts

In [ ]:
sns.countplot(diamonds['carat'])

In [ ]:
sns.distplot(diamonds['carat'])

In [ ]:
sns.distplot(diamonds['carat'],kde=False, bins=np.arange(0.2,2.05,.05).tolist())

In [ ]:
sns.boxplot(diamonds['carat'])

In [ ]:
sns.violinplot(data=diamonds,x='carat',y='price')

In [ ]:
sns.scatterplot(data=diamonds.query('y < 20 & x > 2.5'),x='x',y='y') #let's fix this graph, deal with the outliers (depends), winsorize everything

In [ ]:
sns.jointplot(data=diamonds.query('y < 20 & x > 2.5'),x='x',y='y')

In [ ]:
# how is carat related to price? 

sns.scatterplot(data=diamonds.query('y < 3000 & x < 1'), x='carat',y='price')

In [ ]:
# how is cut related to price?

sns.scatterplot(data=diamonds, x='cut',y='price')

In [ ]:
sns.boxplot(data=diamonds.query('price < 6000'),x='cut',y='price')

In [ ]:
sns.distplot(diamonds.query('price<5000')['price'],kde=False,bins=250)

In [ ]:
# how is carat related to price? ; kind=resid, reg, kde, etc.

sns.jointplot(data=diamonds.query('carat<2 & price<10000'),y='price',x='carat',kind='reg')

# 

In [7]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr # you might need to install this (see above)
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# don't copy and paste these lists and number in several places 
# (which is what the original "simple program" did... that's bad programming!)
# instead, create a var - if we change it here, it changes everywhere
stocks = ['AAPL','MSFT','VZ']
start_yr = 2000

# load
stock_prices = pdr.get_data_yahoo(stocks, start=start_yr)
stock_prices = stock_prices.filter(like='Adj Close') # reduce to just columns with this in the name
stock_prices.columns = stocks # put their tickers as column names
stock_prices # print

,AAPL,MSFT,VZ
Date,,,
2000-01-03,3.470226,37.495686,20.721500
2000-01-04,3.177650,36.229057,20.051672
2000-01-05,3.224152,36.611080,20.721500
2000-01-06,2.945139,35.384666,20.631170
2000-01-07,3.084645,35.847076,20.478834
...,...,...,...
2020-02-07,320.029999,183.889999,59.939999
2020-02-10,321.549988,188.699997,60.240002
2020-02-11,319.609985,184.440002,58.689999


In [8]:
# this is wide data... so if we want to create a new variable, we have to do it once for each firm...
# what if we have 1000 firms? seems tough to do...
# make long/tidy:
stock_prices = stock_prices.stack().swaplevel().sort_index().reset_index()
stock_prices.columns = ['Firm','Date','Adj Close']
stock_prices # print - now that is formatted nicely, like CRSP! 

# note: we might have to tweak the regression section of "Simple Program"
# to work with the long data, but I suspect that it would be shorter code and 
# more "generally applicable" (ie easily work with N=1000+ firms)

,Firm,Date,Adj Close
0,AAPL,2000-01-03,3.470226
1,AAPL,2000-01-04,3.177650
2,AAPL,2000-01-05,3.224152
3,AAPL,2000-01-06,2.945139
4,AAPL,2000-01-07,3.084645
...,...,...,...
15178,VZ,2020-02-07,59.939999
15179,VZ,2020-02-10,60.240002
15180,VZ,2020-02-11,58.689999
15181,VZ,2020-02-12,58.230000


In [9]:
# add return var.
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby
stock_prices['ret'] = stock_prices.groupby('Firm')['Adj Close'].pct_change()
stock_prices # print - the first ret for each firm should be missing...b

,Firm,Date,Adj Close,ret
0,AAPL,2000-01-03,3.470226,NaN
1,AAPL,2000-01-04,3.177650,-0.084310
2,AAPL,2000-01-05,3.224152,0.014634
3,AAPL,2000-01-06,2.945139,-0.086538
4,AAPL,2000-01-07,3.084645,0.047368
...,...,...,...,...
15178,VZ,2020-02-07,59.939999,0.008412
15179,VZ,2020-02-10,60.240002,0.005005
15180,VZ,2020-02-11,58.689999,-0.025730
15181,VZ,2020-02-12,58.230000,-0.007838


In [10]:
stock_prices['ret'].isna().sum()

3

In [11]:
stock_prices.groupby('Firm')     ['ret'].describe() #for each firm, describe the return variable

,count,mean,std,min,25%,50%,75%,max
Firm,,,,,,,,
AAPL,5060.0,0.001235,0.025351,-0.518692,-0.010326,0.000899,0.012946,0.139049
MSFT,5060.0,0.000494,0.018967,-0.155977,-0.008045,0.000337,0.008933,0.195652
VZ,5060.0,0.000323,0.015350,-0.118461,-0.007238,0.000395,0.007704,0.146324


In [ ]:
help(stock_prices['ret'].isnull)